# Week 8 Assignment

src: <https://almondine-song-c43.notion.site/Homework-7-QAP-9142687e4ff94bda967f088a36cff41d>

## Problem 1

Let φ be the transformation of a column vector into a polynomial like we discussed in class (using lagrange interpolation over the x values [0, 1, …, n] and the y values being the values in the vector).

Use Python compute:

$$
\phi(c\cdot\begin{bmatrix}x_1\\x_2\\x_3\end{bmatrix}) = c\cdot\phi(\begin{bmatrix}x_1\\x_2\\x_3\end{bmatrix})
$$

Test out a few vectors to convince yourself this is true in general.

In English, what is the above equality stating?

## Ans

In [31]:
from scipy.interpolate import lagrange
import numpy as np
from numpy.polynomial import polynomial as P, Polynomial as Poly

def vecToPoly(vec):
    p1d = lagrange(list(range(len(vec))), vec)
    return Poly(p1d.coef[::-1])

# Sample test cases
vec = [8, 10, 4, 6]
c = 7

poly1 = vecToPoly(vec) * c
poly2 = vecToPoly(np.array(vec) * c)

np.testing.assert_array_almost_equal_nulp(poly1.coef, poly2.coef)

What this means is that multiplying a scalar to a vector is homomorphic to multiplying the same scalar to a polynomial generated from the vector using Lagrange interpolation.

## Problem 2: QAP by hand

Convert the following R1CS into a QAP **over real numbers, not a finite field**

```python
import numpy as np
import random

# Define the matrices
A = np.array([[0,0,3,0,0,0],
               [0,0,0,0,1,0],
               [0,0,1,0,0,0]])

B = np.array([[0,0,1,0,0,0],
               [0,0,0,1,0,0],
               [0,0,0,5,0,0]])

C = np.array([[0,0,0,0,1,0],
               [0,0,0,0,0,1],
               [-3,1,1,2,0,-1]])

# pick values for x and y
x = 100
y = 100

# this is our orignal formula
out = (3 * x * x * y) + (5 * x * y) - x - (2 * y) + 3
v1 = 3 * x * x
v2 = v1 * y
# the witness vector with the intermediate variables inside
w = np.array([1, out, x, y, v1, v2])

result = C.dot(w) == np.multiply(A.dot(w),B.dot(w))
assert result.all(), "result contains an inequality"
```

You can use a computer (Python, sage, etc) to check your work at each step and do the Lagrange interpolate, but you must show each step.

**Be sure to check the polynomials manually because you will get precision loss when interpolating over floats/real numbers.**

Check your work by seeing that the polynomial on both sides of the equation is the same.

## Ans

In [86]:
# We have Ls . Rs = Os
# We want to convert into Uw . Vw = Ww + h(x)t(x) where Uw, Vw, and Ww are polynomials

import numpy as np
from scipy.interpolate import lagrange

def transformWithLagrange(v):
    rows, cols = v.shape
    ret = np.zeros((cols, rows))
    for i in range(0, cols):
        col = lagrange(list(range(1, rows + 1)), v[:,i]).coef
        ret[i] = col
    ret = ret.transpose()
    return ret

# Define the matrices
L = np.array([[0,0,3,0,0,0],
               [0,0,0,0,1,0],
               [0,0,1,0,0,0]])

R = np.array([[0,0,1,0,0,0],
               [0,0,0,1,0,0],
               [0,0,0,5,0,0]])

O = np.array([[0,0,0,0,1,0],
               [0,0,0,0,0,1],
               [-3,1,1,2,0,-1]])    

# Converting L to U
U = transformWithLagrange(L)
print(f"U: {U}")

# Converting R to V
V = transformWithLagrange(R)
print(f"V: {V}")

# Converting O to W
W = transformWithLagrange(O)
print(f"W: {W}")

# witness
x = 100
y = 100
v1 = 3*x*x
v2 = v1*y
out = v2 + 5*x*y - x - 2 * y + 3
witness = np.array([1, out, x, y, v1, v2])

# Uw . Vw = Ww + h(x)t(x)
Uw = np.poly1d(np.matmul(U, witness))
Vw = np.poly1d(np.matmul(V, witness))
Ww = np.poly1d(np.matmul(W, witness))

t = np.poly1d([1, -1]) \
    * np.poly1d([1, -2]) \
    * np.poly1d([1, -3])
h = (Uw * Vw - Ww)/t

print(f"Uw: \n{Uw}")
print(f"Vw: \n{Vw}")
print(f"Ww: \n{Ww}")
print(f"t: \n{t}")
print(f"h: \n{h}")

U: [[ 0.  0.  2.  0. -1.  0.]
 [ 0.  0. -9.  0.  4.  0.]
 [ 0.  0. 10.  0. -3.  0.]]
V: [[ 0.   0.   0.5  1.5  0.   0. ]
 [ 0.   0.  -2.5 -3.5  0.   0. ]
 [ 0.   0.   3.   2.   0.   0. ]]
W: [[-1.5  0.5  0.5  1.   0.5 -1.5]
 [ 4.5 -1.5 -1.5 -3.  -2.5  5.5]
 [-3.   1.   1.   2.   3.  -4. ]]
Uw: 
           2
-2.98e+04 x + 1.191e+05 x - 8.9e+04
Vw: 
     2
200 x - 600 x + 500
Ww: 
           2
-2.96e+06 x + 1.185e+07 x - 8.86e+06
t: 
   3     2
1 x - 6 x + 11 x - 6
h: 
(poly1d([-5960000.,  5940000.]), poly1d([0.]))


## Problem 3: QAP over a finite field

Refer to the code here: https://www.rareskills.io/post/r1cs-to-qap

Do the same operation R1CS above but convert it to a QAP over a finite field. Don’t do it by hand, use Python. If you pick GF79 like the article does, you’ll need to find the congruent element in the field since some of the scalar values for the R1CS above will be negative or be larger than 79.

## Ans

In [74]:
import numpy as np

w = np.empty([3 , 4])
w

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])